In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [5]:
DATASET_PATH = "/kaggle/input/brain-tumor-mri-dataset/Training"


In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 25
LR = 1e-4
NUM_CLASSES = 4


In [7]:
train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(0, translate=(0.05,0.05)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

test_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])


In [8]:
full_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_tf)

print("Classes:", full_dataset.classes)
print("Total images:", len(full_dataset))


Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
Total images: 5712


In [9]:
train_size = int(0.8 * len(full_dataset))
val_size   = int(0.1 * len(full_dataset))
test_size  = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(
    full_dataset, [train_size, val_size, test_size]
)

val_ds.dataset.transform = test_tf
test_ds.dataset.transform = test_tf


In [10]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)


In [11]:
class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()

        self.channel_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

        self.spatial_att = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel Attention
        ca = self.channel_att(x)
        x = x * ca

        # Spatial Attention
        avg = torch.mean(x, dim=1, keepdim=True)
        max_, _ = torch.max(x, dim=1, keepdim=True)
        sa = self.spatial_att(torch.cat([avg, max_], dim=1))

        return x * sa


In [12]:
class ResNet50_CBAM(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.backbone = models.resnet50(pretrained=True)

        self.cbam = CBAM(2048)

        self.backbone.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.cbam(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.backbone.fc(x)
        return x


In [13]:
model = ResNet50_CBAM(NUM_CLASSES).to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 242MB/s]


In [14]:
class_counts = torch.bincount(torch.tensor(full_dataset.targets))
class_weights = 1. / class_counts.float()
class_weights = class_weights.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=LR)


In [15]:
def train_epoch(model, loader):
    model.train()
    correct = total = 0

    for x, y in tqdm(loader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(out, 1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return correct / total


In [16]:
def evaluate(model, loader):
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            _, preds = torch.max(out, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total


In [17]:
for epoch in range(EPOCHS):
    train_acc = train_epoch(model, train_loader)
    val_acc = evaluate(model, val_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}]")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


100%|██████████| 143/143 [01:04<00:00,  2.23it/s]


Epoch [1/25]
Train Acc: 0.8796 | Val Acc: 0.9440


100%|██████████| 143/143 [00:44<00:00,  3.20it/s]


Epoch [2/25]
Train Acc: 0.9713 | Val Acc: 0.9650


100%|██████████| 143/143 [00:44<00:00,  3.21it/s]


...

Epoch [23/25]
Train Acc: 0.9954 | Val Acc: 0.9860


100%|██████████| 143/143 [00:42<00:00,  3.35it/s]


Epoch [24/25]
Train Acc: 0.9989 | Val Acc: 0.9912


100%|██████████| 143/143 [00:42<00:00,  3.36it/s]


Epoch [25/25]
Train Acc: 0.9982 | Val Acc: 0.9895


In [18]:
test_acc = evaluate(model, test_loader)
print("Test Accuracy:", test_acc)


Test Accuracy: 0.9895104895104895


In [ ]:
import numpy as np
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score
)
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
model.eval()

all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)

        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(probs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())
        all_probs.extend(probs.cpu().numpy())


In [ ]:
all_preds  = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs  = np.array(all_probs)


In [ ]:
print(
    classification_report(
        all_labels,
        all_preds,
        target_names=full_dataset.classes,
        digits=4
    )
)


In [ ]:
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(6,5))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=full_dataset.classes,
    yticklabels=full_dataset.classes
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
auc_macro = roc_auc_score(
    all_labels,
    all_probs,
    multi_class="ovr",
    average="macro"
)

auc_weighted = roc_auc_score(
    all_labels,
    all_probs,
    multi_class="ovr",
    average="weighted"
)

print("ROC-AUC (Macro):", auc_macro)
print("ROC-AUC (Weighted):", auc_weighted)


In [ ]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate(self, x, class_idx):
        self.model.zero_grad()
        output = self.model(x)
        score = output[:, class_idx]
        score.backward()

        gradients = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (gradients * self.activations).sum(dim=1)
        cam = torch.relu(cam)

        cam = cam - cam.min()
        cam = cam / cam.max()

        return cam.detach().cpu().numpy()


In [ ]:
target_layer = model.cbam
gradcam = GradCAM(model, target_layer)


In [ ]:
def show_gradcam(image, cam, label):
    image = image.permute(1,2,0).cpu().numpy()
    image = (image - image.min()) / (image.max() - image.min())

    cam = cam[0]
    cam = np.uint8(255 * cam)

    plt.figure(figsize=(5,5))
    plt.imshow(image)
    plt.imshow(cam, cmap="jet", alpha=0.5)
    plt.title(label)
    plt.axis("off")
    plt.show()


In [ ]:
image, label = test_ds[0]
input_img = image.unsqueeze(0).to(device)

cam = gradcam.generate(input_img, label)

show_gradcam(image, cam, full_dataset.classes[label])
